In [29]:
#concatenate all files with pure Python

import glob

read_files = glob.glob("*/*")

with open("result.csv", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())


In [30]:
# data carpetry with Pandas 
import pandas as pd

#renaming columns
#write skipinitialspace=True to strip whitespace from each string

df = pd.read_csv("result.csv", header=None, skipinitialspace=True, names=['session', 'student_Id', 'exercise', 'activity',
'start_time', 'end_time', 'idle_time', 'mouse_wheel', 'mouse_wheel_click', 'mouse_click_left', 'mouse_click_right', 
'mouse_movement', 'keystroke'])

df.head(3)

,session,student_Id,exercise,activity,start_time,end_time,idle_time,mouse_wheel,mouse_wheel_click,mouse_click_left,mouse_click_right,mouse_movement,keystroke
0,1,1,Es,Other,2.10.2014 11:25:33,2.10.2014 11:25:34,0,0,0,0,0,84,0
1,1,1,Es,Aulaweb,2.10.2014 11:25:35,2.10.2014 11:25:42,218,0,0,4,0,397,0
2,1,1,Es,Blank,2.10.2014 11:25:43,2.10.2014 11:25:43,0,0,0,0,0,59,0


In [31]:
#split start time and start date

df['start_date'] = df['start_time'].str.split(' ').str.get(0) 
df['start_time'] = df['start_time'].str.split(' ').str.get(1) 


In [32]:
#split end time and end date

df['end_date'] = df['end_time'].str.split(' ').str.get(0) 
df['end_time'] = df['end_time'].str.split(' ').str.get(1) 

In [33]:
#write all  data to a new file
clean = df.to_csv('clean_data.csv', index=False)

In [34]:
#group data by session, student_Id, exercise
grouped = df.groupby(['session','student_Id', 'exercise']).aggregate(sum).reset_index()

grouped.head(3)

,session,student_Id,exercise,idle_time,mouse_wheel,mouse_wheel_click,mouse_click_left,mouse_click_right,mouse_movement,keystroke
0,1,1,Es,172151,13,0,34,0,5073,14
1,1,1,Es_1_1,47286517,173,0,1445,131,101233,406
2,1,1,Es_1_2,7688814,153,0,1279,85,101405,342


In [35]:
#write all grouped data to a new file
clean = df.to_csv('grouped_data.csv', index=False)

In [36]:
#load final_grades 1st exam
df1 = pd.read_csv("final_grades_ex1.csv")

#rename columns
df1.columns = ['student_Id', 'ex_1_es_1_1', 'ex_1_es_1_2', 'ex_1_es_2_1', 'ex_1_es_2_2', 'ex_1_es_3_1', 'ex_1_es_3_2', 
              'ex_1_es_3_3', 'ex_1_es_3_4', 'ex_1_es_3_5', 'ex_1_es_4_1', 'ex_1_es_4_2', 'ex_1_es_5_1', 'ex_1_es_5_2',
              'ex_1_es_5_3', 'ex_1_es_6_1', 'ex_1_es_6_2', 'ex_1_total']

In [37]:
#load final_grades 2nd exam
df2 = pd.read_csv("final_grades_ex2.csv")

#rename columns
df2.columns = ['student_Id', 'ex_2_es_1_1', 'ex_2_es_1_2', 'ex_2_es_2_1', 'ex_2_es_2_2', 'ex_2_es_3_1', 'ex_2_es_3_2', 
              'ex_2_es_3_3', 'ex_2_es_3_4', 'ex_2_es_3_5', 'ex_2_es_4_1', 'ex_2_es_4_2', 'ex_2_es_5_1', 'ex_2_es_5_2',
              'ex_2_es_5_3', 'ex_2_es_6_1', 'ex_2_es_6_2', 'ex_2_total']

In [38]:
#load intermediate grades
df3 = pd.read_csv("intermediate_grades.csv")

#rename columns
df3.columns = ['student_Id', 'intermediate_session_2', 'intermediate_session_3', 'intermediate_session_4', 
               'intermediate_session_5', 'intermediate_session_6']

In [39]:
#merging final exam data
merged_ex = pd.merge(left=df1,right=df2, left_on='student_Id', right_on='student_Id')

#check
merged_ex.head(3)

,student_Id,ex_1_es_1_1,ex_1_es_1_2,ex_1_es_2_1,ex_1_es_2_2,ex_1_es_3_1,ex_1_es_3_2,ex_1_es_3_3,ex_1_es_3_4,ex_1_es_3_5,...,ex_2_es_3_4,ex_2_es_3_5,ex_2_es_4_1,ex_2_es_4_2,ex_2_es_5_1,ex_2_es_5_2,ex_2_es_5_3,ex_2_es_6_1,ex_2_es_6_2,ex_2_total
0,7,2,3,1,1.5,1,2,0,0,3.0,...,2,3,15,10,1,2.5,0,20,12.0,78.0
1,20,2,3,2,0.0,1,2,2,2,1.5,...,2,3,15,10,2,8.0,3,25,14.5,97.5
2,24,2,3,1,1.5,1,2,2,2,3.0,...,2,3,15,4,2,10.0,3,15,5.0,70.5


In [40]:
#merging all grades
merged_all = pd.merge(left=merged_ex, right=df3, on='student_Id', how='outer')

#changing NaN to 0
merged_all.fillna(0,inplace=True)

#check
merged_all

,student_Id,ex_1_es_1_1,ex_1_es_1_2,ex_1_es_2_1,ex_1_es_2_2,ex_1_es_3_1,ex_1_es_3_2,ex_1_es_3_3,ex_1_es_3_4,ex_1_es_3_5,...,ex_2_es_5_2,ex_2_es_5_3,ex_2_es_6_1,ex_2_es_6_2,ex_2_total,intermediate_session_2,intermediate_session_3,intermediate_session_4,intermediate_session_5,intermediate_session_6
0,7,2.0,3,1.0,1.5,1,2,0,0.0,3.0,...,2.5,0.0,20.0,12.0,78.0,4.0,4.0,4.5,4.0,2.00
1,20,2.0,3,2.0,0.0,1,2,2,2.0,1.5,...,8.0,3.0,25.0,14.5,97.5,3.0,3.0,4.5,3.5,2.00
2,24,2.0,3,1.0,1.5,1,2,2,2.0,3.0,...,10.0,3.0,15.0,5.0,70.5,2.5,2.0,4.5,3.5,3.00
3,30,0.0,0,2.0,0.0,0,0,0,0.0,0.0,...,5.0,1.5,4.0,5.0,55.5,3.0,2.0,4.5,4.0,2.75
4,37,2.0,3,1.0,0.0,0,2,0,2.0,0.0,...,0.0,0.0,0.0,0.0,30.0,0.0,0.0,4.5,3.5,3.00
5,39,2.0,3,2.0,0.0,0,0,0,2.0,0.0,...,8.0,2.5,24.0,13.0,83.5,3.0,1.0,4.5,3.5,2.50
6,44,2.0,3,1.0,2.0,1,2,2,2.0,3.0,...,10.0,3.0,24.0,13.5,97.5,5.5,4.0,4.5,4.0,3.00
7,46,2.0,3,1.0,0.0,1,2,0,0.0,0.0,...,0.0,0.0,5.0,0.0,22.0,2.0,0.5,4.0,4.0,1.00
8,51,1.5,0,0.0,0.0,1,2,2,2.0,0.0,...,0.0,0.0,0.0,0.0,9.0,1.0,1.0,4.0,4.0,0.50
9,56,2.0,3,2.0,2.0,1,2,2,0.0,0.0,...,10.0,3.0,15.0,9.0,84.0,2.5,3.0,4.5,3.0,3.00


In [41]:
#write all grades to a new file
clean = df.to_csv('all_grades.csv', index=False)

